In [1]:
import Augmentor as aug
import glob
import os
import numpy as np
import cv2 as cv
import PIL
from Augmentor.Operations import Operation

In [2]:
class Lightning(Operation):
    def __init__(self, probability, intensity_low=0.7, intensity_high=1.2):
        Operation.__init__(self, probability)
        self.intensity_low = intensity_low
        self.intensity_high = intensity_high

    def perform_operation(self, images):
        for i, image in enumerate(images):
            image = np.array(image.convert('RGB'))
            row, col, _ = image.shape
            light_intensity = np.random.randint(
                int(self.intensity_low * 100),
                int(self.intensity_high * 100)
            )
            light_intensity /= 100

            gaussian = 100 * np.random.random((row, col, 1))
            gaussian = np.array(gaussian, dtype=np.uint8)
            gaussian = np.concatenate((gaussian, gaussian, gaussian), axis=2)

            image = cv.addWeighted(image, light_intensity, gaussian, 0.25, 0)

            image = PIL.Image.fromarray(image)
            images[i] = image
            
        return images

In [3]:
DIRECTORIES = 'dataset-split/*/*'
folders = []
for f in glob.glob(DIRECTORIES):
    print(f[14:16])
    if os.path.isdir(f):
        if(f[14:16] == "va"):
            folders.append(os.path.abspath(f)+"_va")
        elif(f[14:16] == "tr"):
            folders.append(os.path.abspath(f)+"_tr")
        elif(f[14:16] == "te"):
            folders.append(os.path.abspath(f)+"_te")

print('Classes found {}'.format([os.path.split(x)[1] for x in folders]))


Classes found ['Wart', 'Melanocytic naevus', 'Lentigo', 'Pyogenic Granuloma', 'Haemangioma', 'Squamous Cell Carcinoma', 'Basal Cell Carcinoma', 'Dermatofibroma', 'seborrhoeic keratosis', 'Malignant Melanoma', 'Actinic Keratosis', 'Intraepithelial Carcinoma']


In [4]:
pipelines = {}
for folder in folders:
    print(folder[:-2])
    pipelines[os.path.split(folder)[1]] = \
        (aug.Pipeline(source_directory=folder[:-3],
            output_directory='resnet_augmented',
            save_format='jpg'
            )
        )

classes_count = []
for p in pipelines.values():
    print("Class '{}' has {} samples".format(
        p.augmentor_images[0].class_label,
        len(p.augmentor_images)
    ))
    classes_count.append(len(p.augmentor_images))

print(classes_count)

Initialised with 102 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Wart/resnet_augmented.Initialised with 332 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Melanocytic naevus/resnet_augmented.Initialised with 81 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Lentigo/resnet_augmented.Initialised with 24 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Pyogenic Granuloma/resnet_augmented.Initialised with 115 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Haemangioma/resnet_augmented.Initialised with 328 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Squamous Cell Carcinoma/resnet_augmented.Initialised with 695 image(s) found.
Output directory set to /home/matheuscosta/Workbench/XAI/dataset/Basal Cell Carcinoma/resnet_augmented.Initialised with 106 image(s) found.
Output directory set to 

In [5]:
lightning = Lightning(probability=0.5)

for p in pipelines.values():
    p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
    p.zoom_random(probability=0.4, percentage_area=0.9)
    p.flip_left_right(probability=0.7)
    p.flip_top_bottom(probability=0.5)
    p.random_distortion(probability=0.8, grid_width=5, grid_height=5, magnitude=15)
    p.add_operation(lightning)
    p.resize(probability=1.0, width=448, height=448)

In [6]:
SAME_SAMPLING = False
MULTIPLIER = 29

for p in pipelines.values():
    if SAME_SAMPLING:
        diff = max(classes_count) - len(p.augmentor_images)
        p.sample((len(p.augmentor_images) + diff)*MULTIPLIER + diff)
    else:
        p.sample(len(p.augmentor_images)*MULTIPLIER)

Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F62E41C19D0>: 100%|██████████| 2958/2958 [00:14<00:00, 198.13 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F62B823A970>: 100%|██████████| 9628/9628 [00:40<00:00, 238.59 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F62DEFD4AC0>: 100%|██████████| 2349/2349 [00:15<00:00, 152.11 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F62E5432A60>: 100%|██████████| 696/696 [00:09<00:00, 73.54 Samples/s] 
Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F62E42AD640>: 100%|██████████| 3335/3335 [00:26<00:00, 125.65 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F62B8F08F70>: 100%|██████████| 9512/9512 [01:07<00:00, 141.55 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=448x448 at 0x7F6298A07670>: 100%|██████████| 20155/20155 [02:14<00:00, 149.58 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=448x4